# Imports

In [1]:
# pip install nfl_data_py

In [2]:
import pandas as pd

In [3]:
import nfl_data_py as nfl

# Data

Some explantions for stats:

**WOPR** is a weighted combination of the share of team targets a player receives and the share of team air yards. The formula is: WOPR = 1.5 × Target Market Share + 0.7 × Air Yards Market Share. Elite wide receivers are seen having 0.70+ WOPRs.

**Expected Points Added (EPA)** is a commonly used advanced statistic in football. In short, this stat measures how well a team performs compared to their expectation on a play-by-play basis. EPA and EPA per Play, along with other stats using EPA, can help us evaluate team and player performance.

**Passing Air Conversion Ratio (PACR)** measures the rate at which air yards are converted to passing yards. The formula is PACR = (passing yards)/(air yards).

**Receiver Air Conversion Ratio (RACR)** measures the rate at which targeted air yards are converted to receiving yards. The formula is RACR = (receiving yards)/(air yards).

**Completion Percentage Over Expectation (CPOE)**: (Completion percentage) - (expected completion percentage) for passers.

**dakota** is the adjusted EPA + CPOE composite based on coefficients which best predict adjusted EPA/play in the following year.

In [4]:
years = [2023]

## Play-By-Play

Returns play-by-play data for the years and columns specified

years : required, list of years to pull data for (earliest available is 1999)

columns : optional, list of columns to pull data for

downcast : optional, converts float64 columns to float32, reducing memory usage by ~30%. Will slow down initial load speed ~50%

cache : optional, determines whether to pull pbp data from github repo or local cache generated by nfl.cache_pbp()

alt_path : optional, required if nfl.cache_pbp() is called using an alternate path to the default cache

In [5]:
columns = None
df_play = nfl.import_pbp_data(years, columns, downcast=True, cache=False, alt_path=None)

2023 done.
Downcasting floats.


In [6]:
df_play.head

<bound method NDFrame.head of        play_id          game_id old_game_id home_team away_team season_type  \
0          1.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
1         39.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
2         55.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
3         77.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
4        102.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
...        ...              ...         ...       ...       ...         ...   
23658   4406.0   2023_09_WAS_NE  2023110506        NE       WAS         REG   
23659   4431.0   2023_09_WAS_NE  2023110506        NE       WAS         REG   
23660   4458.0   2023_09_WAS_NE  2023110506        NE       WAS         REG   
23661   4485.0   2023_09_WAS_NE  2023110506        NE       WAS         REG   
23662   4507.0   2023_09_WAS_NE  2023110506        NE       WAS         REG   

       week posteam p

In [7]:
# returns list of columns available in play-by-play dataset
nfl.see_pbp_cols()

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       ...
       'out_of_bounds', 'home_opening_kickoff', 'qb_epa', 'xyac_epa',
       'xyac_mean_yardage', 'xyac_median_yardage', 'xyac_success', 'xyac_fd',
       'xpass', 'pass_oe'],
      dtype='object', length=372)

In [8]:
play = df_play.iloc[1]
for i in range(0, len(play), 10):
    print(play[i: i + 10])
    print()

play_id                    39.0
game_id         2023_01_ARI_WAS
old_game_id          2023091007
home_team                   WAS
away_team                   ARI
season_type                 REG
week                          1
posteam                     WAS
posteam_type               home
defteam                     ARI
Name: 1, dtype: object

side_of_field                       ARI
yardline_100                       35.0
game_date                    2023-09-10
quarter_seconds_remaining         900.0
half_seconds_remaining           1800.0
game_seconds_remaining           3600.0
game_half                         Half1
quarter_end                         0.0
drive                               1.0
sp                                  0.0
Name: 1, dtype: object

qtr                                                           1.0
down                                                          NaN
goal_to_go                                                    0.0
time                              

## Weekly Data

Returns weekly data for the years and columns specified. The weekly data is by player.

years : required, list of years to pull data for (earliest available is 1999)

columns : optional, list of columns to pull data for

downcast : converts float64 columns to float32, reducing memory usage by ~30%. Will slow down initial load speed ~50%

In [9]:
columns = None
df_weekly = nfl.import_weekly_data(years, columns, downcast=True)

Downcasting floats.


In [10]:
df_weekly.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NYJ,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00
1,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,4,REG,...,0.0,0.483465,0,0.0,0.031250,-0.012397,0.038197,0.0,0.80,1.80
2,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,7,REG,...,1.0,1.437224,0,3.2,0.034483,0.094340,0.117762,0.0,1.60,2.60
3,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/private/f_aut...,LA,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,14.46,14.46
4,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/private/f_aut...,LA,2023,2,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,13.98,13.98


In [11]:
play = df_weekly.iloc[0]
for i in range(0, len(play), 10):
    print(play[i: i + 10])
    print()

player_id                                                     00-0023459
player_name                                                    A.Rodgers
player_display_name                                        Aaron Rodgers
position                                                              QB
position_group                                                        QB
headshot_url           https://static.www.nfl.com/image/private/f_aut...
recent_team                                                          NYJ
season                                                              2023
week                                                                   1
season_type                                                          REG
Name: 0, dtype: object

opponent_team         BUF
completions             0
attempts                1
passing_yards         0.0
passing_tds             0
interceptions         0.0
sacks                 1.0
sack_yards           10.0
sack_fumbles            0
sack_fumbles

In [12]:
# each row represents the performance of that player for the week
for row in df_weekly.itertuples():
    print(row.Index, row.player_name)

0 A.Rodgers
1 M.Lewis
2 M.Lewis
3 M.Stafford
4 M.Stafford
5 M.Stafford
6 M.Stafford
7 M.Stafford
8 M.Stafford
9 M.Stafford
10 M.Stafford
11 B.Hoyer
12 B.Hoyer
13 J.Graham
14 J.Graham
15 J.Jones
16 J.Jones
17 B.Gabbert
18 A.Dalton
19 A.Dalton
20 R.Cobb
21 R.Cobb
22 R.Cobb
23 R.Cobb
24 R.Cobb
25 R.Cobb
26 T.Taylor
27 T.Taylor
28 T.Taylor
29 T.Taylor
30 T.Taylor
31 T.Taylor
32 J.Hekker
33 R.Wilson
34 R.Wilson
35 R.Wilson
36 R.Wilson
37 R.Wilson
38 R.Wilson
39 R.Wilson
40 R.Wilson
41 M.Jones
42 M.Jones
43 M.Jones
44 M.Jones
45 K.Cousins
46 K.Cousins
47 K.Cousins
48 K.Cousins
49 K.Cousins
50 K.Cousins
51 K.Cousins
52 K.Cousins
53 B.Anger
54 R.Tannehill
55 R.Tannehill
56 R.Tannehill
57 R.Tannehill
58 R.Tannehill
59 R.Tannehill
60 K.Juszczyk
61 K.Juszczyk
62 K.Juszczyk
63 K.Juszczyk
64 K.Juszczyk
65 A.Thielen
66 A.Thielen
67 A.Thielen
68 A.Thielen
69 A.Thielen
70 A.Thielen
71 A.Thielen
72 A.Thielen
73 Z.Ertz
74 Z.Ertz
75 Z.Ertz
76 Z.Ertz
77 Z.Ertz
78 Z.Ertz
79 Z.Ertz
80 M.Goodwin
81 M.Goodwin

## Seasonal Data

Returns seasonal data, including various calculated market share stats specific to receivers

years (List[int]) : required, list of years to pull data for (earliest available is 1999)

s_type (str) : optional (default 'REG') season type to include in average ('ALL','REG','POST')

In [13]:
df_seasonal = nfl.import_seasonal_data(years)

In [14]:
df_seasonal.head

<bound method NDFrame.head of       player_id  season season_type  completions  attempts  passing_yards  \
0    00-0023459    2023         REG            0         1            0.0   
1    00-0024243    2023         REG            0         0            0.0   
2    00-0026498    2023         REG          166       278         2070.0   
3    00-0026625    2023         REG           23        42          231.0   
4    00-0027696    2023         REG            0         0            0.0   
..          ...     ...         ...          ...       ...            ...   
496  00-0039150    2023         REG          161       252         1375.0   
497  00-0039152    2023         REG           41        68          500.0   
498  00-0039163    2023         REG          173       279         2270.0   
499  00-0039164    2023         REG           50        84          577.0   
500  00-0039165    2023         REG            0         0            0.0   

     passing_tds  interceptions  sacks  sack_

In [15]:
play = df_seasonal.iloc[0]
for i in range(0, len(play), 10):
    print(play[i: i + 10])
    print()

player_id        00-0023459
season                 2023
season_type             REG
completions               0
attempts                  1
passing_yards           0.0
passing_tds               0
interceptions           0.0
sacks                   1.0
sack_yards             10.0
Name: 0, dtype: object

sack_fumbles                       0
sack_fumbles_lost                  0
passing_air_yards               17.0
passing_yards_after_catch        0.0
passing_first_downs              0.0
passing_epa                 -2.03196
passing_2pt_conversions            0
pacr                             0.0
dakota                           0.0
carries                            0
Name: 0, dtype: object

rushing_yards              0.0
rushing_tds                  0
rushing_fumbles            0.0
rushing_fumbles_lost       0.0
rushing_first_downs        0.0
rushing_epa                0.0
rushing_2pt_conversions      0
receptions                   0
targets                      0
receiving_yards        

## Season Rosters

Returns yearly roster information for the seasons specified

years : required, list of years to pull data for (earliest available is 1999)

columns : optional, list of columns to pull data for

In [16]:
columns = ["player_name", "player_id", "position"]
df_roster = nfl.import_seasonal_rosters(years, columns)

In [17]:
df_roster.iloc[0]

player_name    Jason Peters
player_id        00-0022531
position                 OL
Name: 0, dtype: object

# Creating Player Map

In [18]:
player_map = dict()
id_map = dict()

In [19]:
for row in df_roster.itertuples():
    player_map[row.player_id] = dict()
    player_map[row.player_id]["name"] = row.player_name
    player_map[row.player_id]["position"] = row.position
    
    id_map[row.player_name] = row.player_id

In [20]:
for row in df_seasonal.itertuples():
    if row.player_id in player_map:
        player_map[row.player_id]["seasonal_df"] = row._asdict()

In [21]:
# each row represents the performance of that player for the week
for row in df_weekly.itertuples():
    if row.player_display_name in id_map:
        player_id = id_map[row.player_display_name]
        if "week_dfs" in player_map[player_id]:
            player_map[player_id]["week_dfs"].append(row._asdict())
        else:
            player_map[player_id]["week_dfs"] = [row._asdict()]

### Travis Kelce

In [22]:
kelce_id = id_map["Travis Kelce"]

In [23]:
player_map[kelce_id]["seasonal_df"]

{'Index': 22,
 'player_id': '00-0030506',
 'season': 2023,
 'season_type': 'REG',
 'completions': 0,
 'attempts': 0,
 'passing_yards': 0.0,
 'passing_tds': 0,
 'interceptions': 0.0,
 'sacks': 0.0,
 'sack_yards': 0.0,
 'sack_fumbles': 0,
 'sack_fumbles_lost': 0,
 'passing_air_yards': 0.0,
 'passing_yards_after_catch': 0.0,
 'passing_first_downs': 0.0,
 'passing_epa': 0.0,
 'passing_2pt_conversions': 0,
 'pacr': 0.0,
 'dakota': 0.0,
 'carries': 0,
 'rushing_yards': 0.0,
 'rushing_tds': 0,
 'rushing_fumbles': 0.0,
 'rushing_fumbles_lost': 0.0,
 'rushing_first_downs': 0.0,
 'rushing_epa': 0.0,
 'rushing_2pt_conversions': 0,
 'receptions': 57,
 'targets': 72,
 'receiving_yards': 597.0,
 'receiving_tds': 4,
 'receiving_fumbles': 0.0,
 'receiving_fumbles_lost': 0.0,
 'receiving_air_yards': 466.0,
 'receiving_yards_after_catch': 280.0,
 'receiving_first_downs': 30.0,
 'receiving_epa': 34.570952970611394,
 'receiving_2pt_conversions': 0,
 'racr': 8.395131179236696,
 'target_share': 1.9754819427

In [24]:
player_map[kelce_id]["week_dfs"][0]

{'Index': 101,
 'player_id': '00-0030506',
 'player_name': 'T.Kelce',
 'player_display_name': 'Travis Kelce',
 'position': 'TE',
 'position_group': 'TE',
 'headshot_url': 'https://static.www.nfl.com/image/private/f_auto,q_auto/league/ryzr3vbragwe50vtj9af',
 'recent_team': 'KC',
 'season': 2023,
 'week': 2,
 'season_type': 'REG',
 'opponent_team': 'JAX',
 'completions': 0,
 'attempts': 0,
 'passing_yards': 0.0,
 'passing_tds': 0,
 'interceptions': 0.0,
 'sacks': 0.0,
 'sack_yards': 0.0,
 'sack_fumbles': 0,
 'sack_fumbles_lost': 0,
 'passing_air_yards': 0.0,
 'passing_yards_after_catch': 0.0,
 'passing_first_downs': 0.0,
 'passing_epa': nan,
 'passing_2pt_conversions': 0,
 'pacr': nan,
 'dakota': nan,
 'carries': 0,
 'rushing_yards': 0.0,
 'rushing_tds': 0,
 'rushing_fumbles': 0.0,
 'rushing_fumbles_lost': 0.0,
 'rushing_first_downs': 0.0,
 'rushing_epa': nan,
 'rushing_2pt_conversions': 0,
 'receptions': 4,
 'targets': 9,
 'receiving_yards': 26.0,
 'receiving_tds': 1,
 'receiving_fumble

# Data by Position Group

In [25]:
qbs = list()
rbs = list()
wrs = list()
tes = list()

In [26]:
for key in player_map:
    player = player_map[key]
    if "seasonal_df" in player:  # not all players have a seasonal df
        if player["position"] == "QB":
            qbs.append(player)
        if player["position"] == "RB":
            rbs.append(player)
        if player["position"] == "WR":
            wrs.append(player)
        if player["position"] == "TE":
            tes.append(player)

In [54]:
df_qb = pd.DataFrame(columns=['Name', 'Passing EPA', 'Rushing EPA', 'Total EPA', 'EPA Per Game', 'Fantasy Points', 'Fantasy Points Per Game', 'Dakota', 'Dakota Per Game', 'PACR', 'PACR Per Game'])
i = 1
for player in qbs:
    print(player["seasonal_df"].keys())
    if player["seasonal_df"]["fantasy_points_ppr"] > -500 and player["seasonal_df"]["games"] > 1:
        print(player["seasonal_df"].keys())
        break
        name = player['name']
        games = player['seasonal_df']['games']
        fantasy_points_ppr = player['seasonal_df']['fantasy_points_ppr']
        dakota = player['seasonal_df']['dakota']
        pacr = player['seasonal_df']['pacr']
        passing_epa = player['seasonal_df']['passing_epa']
        rushing_epa = player['seasonal_df']['rushing_epa']
        total_epa = passing_epa + rushing_epa
        
        df_qb.loc[i] = [name, passing_epa, rushing_epa, total_epa, total_epa / games, fantasy_points_ppr, fantasy_points_ppr / games, dakota, dakota / games, pacr, pacr / games]
        i += 1
    break

dict_keys(['Index', 'player_id', 'season', 'season_type', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share', 'wopr_x', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr', 'games', 'tgt_sh', 'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh', 'dom', 'w8dom', 'yptmpa', 'ppr_sh'])


In [28]:
names = ["Brock Purdy", "C.J. Stroud", "Trevor Lawrence", "Justin Herbert", "Justin Fields"]
def highlight_names(row):
    """"""
    if row['Name'] in names:
        return ['background-color: yellow']*len(row)
    return ['']*len(row)

In [29]:
df_qb

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
1,Matthew Stafford,10.725585,8.292132,19.017716,2.377215,111.60,13.950000,0.368248,0.046031,7.318930,0.914866
2,Brian Hoyer,-8.780773,-1.815346,-10.596118,-5.298059,4.94,2.470000,0.184016,0.092008,1.425893,0.712946
3,Andy Dalton,4.548330,1.428460,5.976790,2.988395,23.64,11.820000,0.063312,0.031656,0.756813,0.378407
4,Tyrod Taylor,-6.304451,0.143401,-6.161050,-1.026842,40.96,6.826667,0.450631,0.075105,3.135817,0.522636
5,Russell Wilson,-7.331117,16.531328,9.200210,1.150026,134.62,16.827500,0.992861,0.124108,7.978740,0.997342
6,Kirk Cousins,37.103045,1.687369,38.790414,4.848802,149.74,18.717500,1.061110,0.132639,8.482994,1.060374
7,Ryan Tannehill,-18.624608,-5.614132,-24.238740,-4.039790,51.12,8.520000,0.507112,0.084519,4.599203,0.766534
8,Geno Smith,-1.745355,-9.907529,-11.652884,-1.456611,98.18,12.272500,0.687580,0.085948,8.067709,1.008464
9,Derek Carr,-3.147008,-15.069247,-18.216256,-2.024028,117.14,13.015556,0.795030,0.088337,7.627194,0.847466
10,Jimmy Garoppolo,-17.026532,-2.711120,-19.737653,-3.289609,64.10,10.683333,0.428102,0.071350,5.848699,0.974783


In [30]:
df_qb.sort_values(by='Fantasy Points', ascending=False).style.apply(highlight_names, axis=1)

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
27,Josh Allen,69.359606,28.142430,97.502036,10.833560,212.220000,23.580000,1.687734,0.187526,8.810733,0.978970
36,Jalen Hurts,44.073286,16.721520,60.794807,6.754979,205.480000,22.831111,1.518848,0.168761,8.157489,0.906388
20,Patrick Mahomes,57.701408,11.890552,69.591960,7.732440,171.480000,19.053333,1.327775,0.147531,10.325359,1.147262
25,Lamar Jackson,17.316286,6.680434,23.996720,2.666302,170.160000,18.906667,1.166127,0.129570,9.949339,1.105482
33,Tua Tagovailoa,84.778906,-26.718125,58.060780,6.451198,165.660000,18.406667,1.502274,0.166919,11.470439,1.274493
42,Sam Howell,-27.489225,15.184313,-12.304913,-1.367213,158.540000,17.615556,0.873351,0.097039,7.975596,0.886177
56,C.J. Stroud,54.169188,3.250421,57.419609,7.177451,156.600000,19.575000,0.965447,0.120681,7.625980,0.953248
22,Joshua Dobbs,-52.535983,12.462707,-40.073276,-4.452586,151.480000,16.831111,0.646637,0.071849,6.973860,0.774873
6,Kirk Cousins,37.103045,1.687369,38.790414,4.848802,149.740000,18.717500,1.061110,0.132639,8.482994,1.060374
35,Justin Herbert,43.270683,2.405487,45.676170,6.525167,149.700000,21.385714,0.775044,0.110721,6.916435,0.988062


In [31]:
df_qb.sort_values(by='Total EPA', ascending=False).style.apply(highlight_names, axis=1)

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
27,Josh Allen,69.359606,28.142430,97.502036,10.833560,212.220000,23.580000,1.687734,0.187526,8.810733,0.978970
43,Brock Purdy,62.380442,13.258625,75.639068,9.454883,138.020000,17.252500,1.629823,0.203728,9.261292,1.157662
20,Patrick Mahomes,57.701408,11.890552,69.591960,7.732440,171.480000,19.053333,1.327775,0.147531,10.325359,1.147262
36,Jalen Hurts,44.073286,16.721520,60.794807,6.754979,205.480000,22.831111,1.518848,0.168761,8.157489,0.906388
33,Tua Tagovailoa,84.778906,-26.718125,58.060780,6.451198,165.660000,18.406667,1.502274,0.166919,11.470439,1.274493
56,C.J. Stroud,54.169188,3.250421,57.419609,7.177451,156.600000,19.575000,0.965447,0.120681,7.625980,0.953248
35,Justin Herbert,43.270683,2.405487,45.676170,6.525167,149.700000,21.385714,0.775044,0.110721,6.916435,0.988062
13,Dak Prescott,33.631372,7.535524,41.166895,5.145862,140.240000,17.530000,1.151329,0.143916,8.444243,1.055530
6,Kirk Cousins,37.103045,1.687369,38.790414,4.848802,149.740000,18.717500,1.061110,0.132639,8.482994,1.060374
14,Jared Goff,33.593030,-8.963110,24.629920,3.078740,138.260000,17.282500,1.057508,0.132188,9.452039,1.181505


In [32]:
df_qb.sort_values(by='EPA Per Game', ascending=False).style.apply(highlight_names, axis=1)

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
27,Josh Allen,69.359606,28.142430,97.502036,10.833560,212.220000,23.580000,1.687734,0.187526,8.810733,0.978970
43,Brock Purdy,62.380442,13.258625,75.639068,9.454883,138.020000,17.252500,1.629823,0.203728,9.261292,1.157662
20,Patrick Mahomes,57.701408,11.890552,69.591960,7.732440,171.480000,19.053333,1.327775,0.147531,10.325359,1.147262
56,C.J. Stroud,54.169188,3.250421,57.419609,7.177451,156.600000,19.575000,0.965447,0.120681,7.625980,0.953248
36,Jalen Hurts,44.073286,16.721520,60.794807,6.754979,205.480000,22.831111,1.518848,0.168761,8.157489,0.906388
35,Justin Herbert,43.270683,2.405487,45.676170,6.525167,149.700000,21.385714,0.775044,0.110721,6.916435,0.988062
33,Tua Tagovailoa,84.778906,-26.718125,58.060780,6.451198,165.660000,18.406667,1.502274,0.166919,11.470439,1.274493
13,Dak Prescott,33.631372,7.535524,41.166895,5.145862,140.240000,17.530000,1.151329,0.143916,8.444243,1.055530
6,Kirk Cousins,37.103045,1.687369,38.790414,4.848802,149.740000,18.717500,1.061110,0.132639,8.482994,1.060374
12,Taylor Heinicke,4.559365,1.648181,6.207546,3.103773,27.120000,13.560000,0.147072,0.073536,1.551759,0.775879


In [33]:
df_qb.sort_values(by='Dakota', ascending=False).style.apply(highlight_names, axis=1)

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
27,Josh Allen,69.359606,28.142430,97.502036,10.833560,212.220000,23.580000,1.687734,0.187526,8.810733,0.978970
43,Brock Purdy,62.380442,13.258625,75.639068,9.454883,138.020000,17.252500,1.629823,0.203728,9.261292,1.157662
36,Jalen Hurts,44.073286,16.721520,60.794807,6.754979,205.480000,22.831111,1.518848,0.168761,8.157489,0.906388
33,Tua Tagovailoa,84.778906,-26.718125,58.060780,6.451198,165.660000,18.406667,1.502274,0.166919,11.470439,1.274493
20,Patrick Mahomes,57.701408,11.890552,69.591960,7.732440,171.480000,19.053333,1.327775,0.147531,10.325359,1.147262
25,Lamar Jackson,17.316286,6.680434,23.996720,2.666302,170.160000,18.906667,1.166127,0.129570,9.949339,1.105482
13,Dak Prescott,33.631372,7.535524,41.166895,5.145862,140.240000,17.530000,1.151329,0.143916,8.444243,1.055530
6,Kirk Cousins,37.103045,1.687369,38.790414,4.848802,149.740000,18.717500,1.061110,0.132639,8.482994,1.060374
14,Jared Goff,33.593030,-8.963110,24.629920,3.078740,138.260000,17.282500,1.057508,0.132188,9.452039,1.181505
5,Russell Wilson,-7.331117,16.531328,9.200210,1.150026,134.620000,16.827500,0.992861,0.124108,7.978740,0.997342


In [34]:
df_qb.sort_values(by='Dakota Per Game', ascending=False).style.apply(highlight_names, axis=1)

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
43,Brock Purdy,62.380442,13.258625,75.639068,9.454883,138.020000,17.252500,1.629823,0.203728,9.261292,1.157662
27,Josh Allen,69.359606,28.142430,97.502036,10.833560,212.220000,23.580000,1.687734,0.187526,8.810733,0.978970
36,Jalen Hurts,44.073286,16.721520,60.794807,6.754979,205.480000,22.831111,1.518848,0.168761,8.157489,0.906388
33,Tua Tagovailoa,84.778906,-26.718125,58.060780,6.451198,165.660000,18.406667,1.502274,0.166919,11.470439,1.274493
20,Patrick Mahomes,57.701408,11.890552,69.591960,7.732440,171.480000,19.053333,1.327775,0.147531,10.325359,1.147262
13,Dak Prescott,33.631372,7.535524,41.166895,5.145862,140.240000,17.530000,1.151329,0.143916,8.444243,1.055530
6,Kirk Cousins,37.103045,1.687369,38.790414,4.848802,149.740000,18.717500,1.061110,0.132639,8.482994,1.060374
14,Jared Goff,33.593030,-8.963110,24.629920,3.078740,138.260000,17.282500,1.057508,0.132188,9.452039,1.181505
25,Lamar Jackson,17.316286,6.680434,23.996720,2.666302,170.160000,18.906667,1.166127,0.129570,9.949339,1.105482
53,Jaren Hall,2.315648,-2.079497,0.236151,0.118076,3.040000,1.520000,0.257416,0.128708,4.332512,2.166256


In [35]:
df_qb.sort_values(by='PACR', ascending=False).style.apply(highlight_names, axis=1)

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
33,Tua Tagovailoa,84.778906,-26.718125,58.060780,6.451198,165.660000,18.406667,1.502274,0.166919,11.470439,1.274493
20,Patrick Mahomes,57.701408,11.890552,69.591960,7.732440,171.480000,19.053333,1.327775,0.147531,10.325359,1.147262
25,Lamar Jackson,17.316286,6.680434,23.996720,2.666302,170.160000,18.906667,1.166127,0.129570,9.949339,1.105482
14,Jared Goff,33.593030,-8.963110,24.629920,3.078740,138.260000,17.282500,1.057508,0.132188,9.452039,1.181505
43,Brock Purdy,62.380442,13.258625,75.639068,9.454883,138.020000,17.252500,1.629823,0.203728,9.261292,1.157662
40,Mac Jones,-40.222925,-5.683796,-45.906721,-5.100747,99.540000,11.060000,0.372609,0.041401,9.163622,1.018180
18,Cooper Rush,-4.763626,-7.393628,-12.157254,-3.039313,0.540000,0.135000,0.000000,0.000000,9.125000,2.281250
27,Josh Allen,69.359606,28.142430,97.502036,10.833560,212.220000,23.580000,1.687734,0.187526,8.810733,0.978970
30,Gardner Minshew,-10.772911,1.768797,-9.004115,-1.125514,91.580000,11.447500,0.771477,0.096435,8.496734,1.062092
6,Kirk Cousins,37.103045,1.687369,38.790414,4.848802,149.740000,18.717500,1.061110,0.132639,8.482994,1.060374


In [36]:
df_qb.sort_values(by='PACR Per Game', ascending=False).style.apply(highlight_names, axis=1)

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
18,Cooper Rush,-4.763626,-7.393628,-12.157254,-3.039313,0.540000,0.135000,0.000000,0.000000,9.125000,2.281250
53,Jaren Hall,2.315648,-2.079497,0.236151,0.118076,3.040000,1.520000,0.257416,0.128708,4.332512,2.166256
57,Anthony Richardson,4.587791,0.430104,5.017896,1.254474,72.680000,18.170000,0.224194,0.056049,5.912336,1.478084
48,Tyson Bagent,-23.398139,12.388887,-11.009252,-2.752313,45.580000,11.395000,0.229295,0.057324,5.870203,1.467551
33,Tua Tagovailoa,84.778906,-26.718125,58.060780,6.451198,165.660000,18.406667,1.502274,0.166919,11.470439,1.274493
14,Jared Goff,33.593030,-8.963110,24.629920,3.078740,138.260000,17.282500,1.057508,0.132188,9.452039,1.181505
38,Justin Fields,-22.823526,9.199304,-13.624222,-2.270704,105.740000,17.623333,0.431034,0.071839,6.960992,1.160165
43,Brock Purdy,62.380442,13.258625,75.639068,9.454883,138.020000,17.252500,1.629823,0.203728,9.261292,1.157662
20,Patrick Mahomes,57.701408,11.890552,69.591960,7.732440,171.480000,19.053333,1.327775,0.147531,10.325359,1.147262
25,Lamar Jackson,17.316286,6.680434,23.996720,2.666302,170.160000,18.906667,1.166127,0.129570,9.949339,1.105482


In [45]:
df_qb.head(1)

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
1,Matthew Stafford,10.725585,8.292132,19.017716,2.377215,111.6,13.95,0.368248,0.046031,7.31893,0.914866


In [52]:
ls

2022 Data.ipynb               __pycache__/
Clustering (Per Game).ipynb   env/
Clustering (Totals).ipynb     images/
FantasyData/                  interactive/
Graph Data.ipynb              main.py
Interactive Clustering.ipynb  models/
NFLData.ipynb                 plotly/
README.md                     utils/


In [51]:
df_qb.to_csv()

,Name,Passing EPA,Rushing EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,Dakota,Dakota Per Game,PACR,PACR Per Game
1,Matthew Stafford,10.725585,8.292132,19.017716,2.377215,111.60,13.950000,0.368248,0.046031,7.318930,0.914866
2,Brian Hoyer,-8.780773,-1.815346,-10.596118,-5.298059,4.94,2.470000,0.184016,0.092008,1.425893,0.712946
3,Andy Dalton,4.548330,1.428460,5.976790,2.988395,23.64,11.820000,0.063312,0.031656,0.756813,0.378407
4,Tyrod Taylor,-6.304451,0.143401,-6.161050,-1.026842,40.96,6.826667,0.450631,0.075105,3.135817,0.522636
5,Russell Wilson,-7.331117,16.531328,9.200210,1.150026,134.62,16.827500,0.992861,0.124108,7.978740,0.997342
6,Kirk Cousins,37.103045,1.687369,38.790414,4.848802,149.74,18.717500,1.061110,0.132639,8.482994,1.060374
7,Ryan Tannehill,-18.624608,-5.614132,-24.238740,-4.039790,51.12,8.520000,0.507112,0.084519,4.599203,0.766534
8,Geno Smith,-1.745355,-9.907529,-11.652884,-1.456611,98.18,12.272500,0.687580,0.085948,8.067709,1.008464
9,Derek Carr,-3.147008,-15.069247,-18.216256,-2.024028,117.14,13.015556,0.795030,0.088337,7.627194,0.847466
10,Jimmy Garoppolo,-17.026532,-2.711120,-19.737653,-3.289609,64.10,10.683333,0.428102,0.071350,5.848699,0.974783


## WRs

In [37]:
df_wr = pd.DataFrame(columns=['Name', 'Target Share', 'Weighted Opportunity X', 'Rushing EPA', 'Receiving EPA', 'Total EPA', 'EPA Per Game', 'Fantasy Points', 'Fantasy Points Per Game', 'RACR', 'RACR Per Game'])
i = 1
for player in wrs:
    if player["seasonal_df"]["fantasy_points_ppr"] > -500 and player["seasonal_df"]["games"] > 1:
        name = player['name']
        games = player['seasonal_df']['games']
        fantasy_points_ppr = player['seasonal_df']['fantasy_points_ppr']
    
        tgt_sh =  player['seasonal_df']['tgt_sh']
        wopr_x = player['seasonal_df']['wopr_x']
        racr = player['seasonal_df']['racr']
        receiving_epa = player['seasonal_df']['receiving_epa']
        rushing_epa = player['seasonal_df']['rushing_epa']
        total_epa = receiving_epa + rushing_epa
        
        df_wr.loc[i] = [name, tgt_sh, wopr_x, rushing_epa, receiving_epa, total_epa, total_epa / games, fantasy_points_ppr, fantasy_points_ppr / games, racr, racr / games]
        i += 1

In [38]:
df_wr

,Name,Target Share,Weighted Opportunity X,Rushing EPA,Receiving EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,RACR,RACR Per Game
1,Julio Jones,0.043478,0.191337,0.000000,1.649969,1.649969,0.824985,9.1,4.550000,1.921053,0.960526
2,Randall Cobb,0.065574,0.901848,0.000000,-8.724053,-8.724053,-1.454009,7.0,1.166667,2.333333,0.388889
3,Marvin Jones,0.074074,0.761398,0.000000,-5.426967,-5.426967,-1.356742,6.5,1.625000,2.454118,0.613529
4,Adam Thielen,0.244373,4.810961,0.236421,40.631195,40.867616,5.108452,149.6,18.700000,9.437467,1.179683
5,Marquise Goodwin,0.040590,1.245247,1.455231,-10.371712,-8.916480,-1.114560,7.3,0.912500,0.365079,0.045635
...,...,...,...,...,...,...,...,...,...,...,...
167,Jonathan Mingo,0.136201,2.865920,0.000000,-7.094804,-7.094804,-1.013543,40.0,5.714286,3.972746,0.567535
168,Zay Flowers,0.237548,4.940719,-0.525382,10.539819,10.014437,1.112715,99.4,11.044444,11.357587,1.261954
169,Rashee Rice,0.120588,2.271900,-2.202130,16.853425,14.651295,1.627922,93.5,10.388889,21.944126,2.438236
170,Puka Nacua,0.303797,6.270180,1.074952,41.106901,42.181853,4.686873,159.1,17.677778,8.610056,0.956673


In [39]:
df_wr.sort_values(by='Fantasy Points Per Game', ascending=False).head(48)

,Name,Target Share,Weighted Opportunity X,Rushing EPA,Receiving EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,RACR,RACR Per Game
28,Tyreek Hill,0.311897,7.180440,1.332226,54.434089,55.766316,6.196257,224.30,24.922222,10.211916,1.134657
78,A.J. Brown,0.301639,7.528059,0.000000,56.459323,56.459323,6.273258,203.50,22.611111,7.546446,0.838494
92,Justin Jefferson,0.259804,3.665702,0.000000,21.564572,21.564572,4.312914,111.10,22.220000,4.803917,0.960783
17,Stefon Diggs,0.299383,6.782239,0.000000,36.164965,36.164965,4.018329,195.40,21.711111,9.427451,1.047495
6,Keenan Allen,0.292490,4.979675,0.153951,37.546931,37.700882,5.385840,148.86,21.265714,5.820356,0.831479
113,Amon-Ra St. Brown,0.294340,4.836002,-1.186781,21.432481,20.245700,2.892243,139.90,19.985714,8.637540,1.233934
96,CeeDee Lamb,0.263736,5.053851,1.497076,50.134433,51.631509,6.453939,158.70,19.837500,9.651071,1.206384
108,Ja'Marr Chase,0.300971,5.947286,-2.737978,37.735006,34.997028,4.374629,157.10,19.637500,8.478459,1.059807
4,Adam Thielen,0.244373,4.810961,0.236421,40.631195,40.867616,5.108452,149.60,18.700000,9.437467,1.179683
170,Puka Nacua,0.303797,6.270180,1.074952,41.106901,42.181853,4.686873,159.10,17.677778,8.610056,0.956673


In [40]:
df_wr.sort_values(by='Total EPA', ascending=False).head(48)

,Name,Target Share,Weighted Opportunity X,Rushing EPA,Receiving EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,RACR,RACR Per Game
78,A.J. Brown,0.301639,7.528059,0.000000,56.459323,56.459323,6.273258,203.50,22.611111,7.546446,0.838494
28,Tyreek Hill,0.311897,7.180440,1.332226,54.434089,55.766316,6.196257,224.30,24.922222,10.211916,1.134657
96,CeeDee Lamb,0.263736,5.053851,1.497076,50.134433,51.631509,6.453939,158.70,19.837500,9.651071,1.206384
170,Puka Nacua,0.303797,6.270180,1.074952,41.106901,42.181853,4.686873,159.10,17.677778,8.610056,0.956673
4,Adam Thielen,0.244373,4.810961,0.236421,40.631195,40.867616,5.108452,149.60,18.700000,9.437467,1.179683
103,Nico Collins,0.189286,4.216066,0.000000,39.162850,39.162850,4.895356,123.10,15.387500,9.344045,1.168006
6,Keenan Allen,0.292490,4.979675,0.153951,37.546931,37.700882,5.385840,148.86,21.265714,5.820356,0.831479
17,Stefon Diggs,0.299383,6.782239,0.000000,36.164965,36.164965,4.018329,195.40,21.711111,9.427451,1.047495
108,Ja'Marr Chase,0.300971,5.947286,-2.737978,37.735006,34.997028,4.374629,157.10,19.637500,8.478459,1.059807
60,D.J. Moore,0.227941,6.041383,0.000000,33.462460,33.462460,3.718051,148.50,16.500000,11.257022,1.250780


In [41]:
df_wr.sort_values(by='RACR', ascending=False).head(48)

,Name,Target Share,Weighted Opportunity X,Rushing EPA,Receiving EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,RACR,RACR Per Game
111,Rondale Moore,0.114983,1.852641,-0.150483,-5.736948,-5.887431,-0.654159,53.80,5.977778,38.685763,4.298418
154,Demario Douglas,0.128028,2.466509,1.080688,4.966770,6.047459,0.755932,52.60,6.575000,26.633282,3.329160
123,Khalil Shakir,0.067138,1.272608,0.000000,17.785859,17.785859,2.223232,46.40,5.800000,23.360849,2.920106
169,Rashee Rice,0.120588,2.271900,-2.202130,16.853425,14.651295,1.627922,93.50,10.388889,21.944126,2.438236
110,Kadarius Toney,0.082353,1.326276,-6.034129,-4.286437,-10.320566,-1.146730,38.20,4.244444,17.797875,1.977542
30,Curtis Samuel,0.139610,2.632084,1.763676,5.808692,7.572368,0.946546,88.20,11.025000,15.154011,1.894251
32,Kendrick Bourne,0.197842,3.723866,0.639064,-9.540425,-8.901362,-1.112670,100.00,12.500000,13.304602,1.663075
91,Darnell Mooney,0.139918,3.265611,-0.762429,-0.776978,-1.539408,-0.192426,57.20,7.150000,11.358334,1.419792
168,Zay Flowers,0.237548,4.940719,-0.525382,10.539819,10.014437,1.112715,99.40,11.044444,11.357587,1.261954
141,Wan'Dale Robinson,0.159420,2.331969,1.179301,11.855622,13.034924,1.862132,55.30,7.900000,11.333960,1.619137


In [42]:
df_wr.sort_values(by='RACR Per Game', ascending=False).head(48)

,Name,Target Share,Weighted Opportunity X,Rushing EPA,Receiving EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,RACR,RACR Per Game
111,Rondale Moore,0.114983,1.852641,-0.150483,-5.736948,-5.887431,-0.654159,53.80,5.977778,38.685763,4.298418
154,Demario Douglas,0.128028,2.466509,1.080688,4.966770,6.047459,0.755932,52.60,6.575000,26.633282,3.329160
90,Quez Watkins,0.070423,0.284645,0.000000,-1.302895,-1.302895,-0.651448,6.10,3.050000,5.901961,2.950980
123,Khalil Shakir,0.067138,1.272608,0.000000,17.785859,17.785859,2.223232,46.40,5.800000,23.360849,2.920106
169,Rashee Rice,0.120588,2.271900,-2.202130,16.853425,14.651295,1.627922,93.50,10.388889,21.944126,2.438236
70,Trenton Irwin,0.120370,0.791905,0.000000,3.467482,3.467482,1.155827,21.10,7.033333,7.016807,2.338936
110,Kadarius Toney,0.082353,1.326276,-6.034129,-4.286437,-10.320566,-1.146730,38.20,4.244444,17.797875,1.977542
30,Curtis Samuel,0.139610,2.632084,1.763676,5.808692,7.572368,0.946546,88.20,11.025000,15.154011,1.894251
32,Kendrick Bourne,0.197842,3.723866,0.639064,-9.540425,-8.901362,-1.112670,100.00,12.500000,13.304602,1.663075
141,Wan'Dale Robinson,0.159420,2.331969,1.179301,11.855622,13.034924,1.862132,55.30,7.900000,11.333960,1.619137


In [43]:
df_wr.sort_values(by='Target Share', ascending=False).head(48)

,Name,Target Share,Weighted Opportunity X,Rushing EPA,Receiving EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,RACR,RACR Per Game
28,Tyreek Hill,0.311897,7.180440,1.332226,54.434089,55.766316,6.196257,224.30,24.922222,10.211916,1.134657
131,Garrett Wilson,0.310502,5.778689,0.491657,0.726223,1.217880,0.173983,98.50,14.071429,5.015313,0.716473
170,Puka Nacua,0.303797,6.270180,1.074952,41.106901,42.181853,4.686873,159.10,17.677778,8.610056,0.956673
78,A.J. Brown,0.301639,7.528059,0.000000,56.459323,56.459323,6.273258,203.50,22.611111,7.546446,0.838494
108,Ja'Marr Chase,0.300971,5.947286,-2.737978,37.735006,34.997028,4.374629,157.10,19.637500,8.478459,1.059807
40,Cooper Kupp,0.300000,3.868306,0.000000,-8.712930,-8.712930,-1.742586,67.40,13.480000,3.485230,0.697046
17,Stefon Diggs,0.299383,6.782239,0.000000,36.164965,36.164965,4.018329,195.40,21.711111,9.427451,1.047495
11,Davante Adams,0.296167,6.566980,0.000000,15.943312,15.943312,1.771479,126.30,14.033333,8.625393,0.958377
113,Amon-Ra St. Brown,0.294340,4.836002,-1.186781,21.432481,20.245700,2.892243,139.90,19.985714,8.637540,1.233934
6,Keenan Allen,0.292490,4.979675,0.153951,37.546931,37.700882,5.385840,148.86,21.265714,5.820356,0.831479


In [44]:
df_wr.sort_values(by='Weighted Opportunity X', ascending=False).head(48)

,Name,Target Share,Weighted Opportunity X,Rushing EPA,Receiving EPA,Total EPA,EPA Per Game,Fantasy Points,Fantasy Points Per Game,RACR,RACR Per Game
78,A.J. Brown,0.301639,7.528059,0.000000,56.459323,56.459323,6.273258,203.50,22.611111,7.546446,0.838494
28,Tyreek Hill,0.311897,7.180440,1.332226,54.434089,55.766316,6.196257,224.30,24.922222,10.211916,1.134657
17,Stefon Diggs,0.299383,6.782239,0.000000,36.164965,36.164965,4.018329,195.40,21.711111,9.427451,1.047495
77,Marquise Brown,0.268293,6.617383,2.173070,1.490775,3.663845,0.407094,112.90,12.544444,5.626698,0.625189
11,Davante Adams,0.296167,6.566980,0.000000,15.943312,15.943312,1.771479,126.30,14.033333,8.625393,0.958377
170,Puka Nacua,0.303797,6.270180,1.074952,41.106901,42.181853,4.686873,159.10,17.677778,8.610056,0.956673
119,Chris Olave,0.250737,6.140157,0.000000,7.023746,7.023746,0.780416,118.30,13.144444,4.717399,0.524155
60,D.J. Moore,0.227941,6.041383,0.000000,33.462460,33.462460,3.718051,148.50,16.500000,11.257022,1.250780
86,Michael Pittman,0.275641,6.015501,0.000000,-4.347483,-4.347483,-0.483054,137.30,15.255556,8.554082,0.950454
108,Ja'Marr Chase,0.300971,5.947286,-2.737978,37.735006,34.997028,4.374629,157.10,19.637500,8.478459,1.059807
